In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import yfinance as yf

In [3]:
pd.set_option('display.max_rows', 100)

In [5]:
acs_df=pd.read_csv('../data/acs_5yr_merged.csv')
zillow_df=pd.read_csv('../data/Zip_Zri_AllHomesPlusMultifamily.csv')

In [7]:
print(acs_df.columns.tolist())

['geo_id', 'do_date', 'nonfamily_households', 'family_households', 'median_year_structure_built', 'rent_burden_not_computed', 'rent_over_50_percent', 'rent_40_to_50_percent', 'rent_35_to_40_percent', 'rent_30_to_35_percent', 'rent_25_to_30_percent', 'rent_20_to_25_percent', 'rent_15_to_20_percent', 'rent_10_to_15_percent', 'rent_under_10_percent', 'total_pop', 'male_pop', 'female_pop', 'median_age', 'white_pop', 'black_pop', 'asian_pop', 'hispanic_pop', 'amerindian_pop', 'other_race_pop', 'two_or_more_races_pop', 'not_hispanic_pop', 'commuters_by_public_transportation', 'households', 'median_income', 'income_per_capita', 'housing_units', 'vacant_housing_units', 'vacant_housing_units_for_rent', 'vacant_housing_units_for_sale', 'median_rent', 'percent_income_spent_on_rent', 'owner_occupied_housing_units', 'million_dollar_housing_units', 'mortgaged_housing_units', 'families_with_young_children', 'two_parent_families_with_young_children', 'two_parents_in_labor_force_families_with_young_chi

In [12]:
acs_df[['geo_id', 'do_date', 'income_less_10000', 'income_10000_14999', 
        'income_15000_19999', 'income_20000_24999', 'income_25000_29999', 
        'income_30000_34999', 'income_35000_39999', 'income_40000_44999', 
        'income_45000_49999', 'income_50000_59999', 'income_60000_74999', 
        'income_75000_99999', 'income_100000_124999', 'income_125000_149999', 
        'income_150000_199999', 'income_200000_or_more']]

,geo_id,do_date,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more
0,601,20072011,2232.0,589.0,408.0,422.0,302.0,278.0,219.0,185.0,162.0,134.0,113.0,122.0,9.0,6.0,12.0,17.0
1,601,20082012,2351.0,585.0,455.0,381.0,333.0,274.0,281.0,126.0,149.0,208.0,121.0,156.0,14.0,5.0,24.0,14.0
2,601,20092013,2659.0,613.0,471.0,374.0,353.0,210.0,274.0,140.0,74.0,233.0,133.0,140.0,39.0,21.0,23.0,23.0
3,601,20102014,2797.0,684.0,407.0,343.0,262.0,263.0,252.0,125.0,113.0,232.0,137.0,102.0,40.0,20.0,20.0,23.0
4,601,20112015,2873.0,716.0,448.0,318.0,262.0,252.0,266.0,118.0,135.0,231.0,138.0,86.0,59.0,22.0,20.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264955,99929,20102014,57.0,89.0,56.0,96.0,55.0,42.0,39.0,101.0,72.0,111.0,101.0,134.0,53.0,41.0,82.0,20.0
264956,99929,20112015,57.0,77.0,54.0,105.0,66.0,44.0,43.0,75.0,51.0,89.0,107.0,152.0,90.0,30.0,43.0,33.0
264957,99929,20122016,38.0,52.0,45.0,112.0,47.0,35.0,48.0,69.0,47.0,110.0,97.0,115.0,106.0,43.0,48.0,25.0
264958,99929,20132017,41.0,53.0,49.0,88.0,76.0,33.0,51.0,54.0,21.0,119.0,107.0,135.0,113.0,76.0,39.0,20.0


In [62]:
zhi_df=pd.read_csv('../data/Zip_Zri_AllHomesPlusMultifamily.csv')
zhi_df=zhi_df.reset_index()
zhi_df_tmp=zhi_df.drop(['RegionID','RegionName','City','State','Metro','CountyName','SizeRank'],axis=1)
zhi_df_pivot=pd.DataFrame(zhi_df_tmp.stack())
zhi_df_pivot=zhi_df_pivot[1:].reset_index()
zhi_df_pivot.columns=['index','Date','Value']
zhi_reshaped=zhi_df_pivot.merge(zhi_df[['index','RegionID','RegionName','City','State','Metro','CountyName','SizeRank']],on='index')
zhi_reshaped=zhi_reshaped.drop('index',axis=1)

In [78]:
zhi_reshaped.dtypes

Date           object
Value         float64
RegionID        int64
RegionName      int64
City           object
State          object
Metro          object
CountyName     object
SizeRank        int64
dtype: object

In [69]:
zipcodes = zhi_reshaped['RegionName'].unique().sort()

In [171]:
zipcodes

array([ 1001,  1002,  1005, ..., 99712, 99801, 99901])

In [143]:
econ_zip_df=pd.read_csv('../../data/CAINC30/CAINC30__ALL_AREAS_1969_2019.csv', low_memory=False)

In [144]:
econ_zip_df.head()
print(econ_zip_df.columns.tolist())

['GeoFIPS', 'GeoName', 'Region', 'TableName', 'LineCode', 'IndustryClassification', 'Description', 'Unit', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


In [145]:
econ_zip_df = econ_zip_df[['GeoFIPS', 'GeoName', 'Region', 'Description', 
             'Unit', '2010', '2011', '2012', '2013', '2014', 
             '2015', '2016', '2017', '2018', '2019']]

In [146]:
pd.unique(econ_zip_df['Region'])

array([' ', '5', '8', '6', '7', '1', '2', '3', '4', nan], dtype=object)

In [147]:
econ_zip_df['GeoName'].unique().shape

(3199,)

In [148]:
econ_zip_df['Description'].unique()

array(['Personal income (thousands of dollars)',
       ' Net earnings by place of residence',
       ' Personal current transfer receipts',
       '  Income maintenance benefits 1/',
       '  Unemployment insurance compensation', '  Retirement and other',
       ' Dividends, interest, and rent 2/', ' Population (persons) 3/',
       ' Per capita personal income 4/', ' Per capita net earnings 4/',
       ' Per capita personal current transfer receipts 4/',
       '  Per capita income maintenance benefits 4/',
       '  Per capita unemployment insurance compensation 4/',
       '  Per capita retirement and other 4/',
       ' Per capita dividends, interest, and rent 4/',
       ' Earnings by place of work', '  Wages and salaries',
       '  Supplements to wages and salaries',
       '   Employer contributions for employee pension and insurance funds 5/',
       '   Employer contributions for government social insurance',
       "  Proprietors' income", "   Farm proprietors' income",
  

In [149]:
econ_zip_df = econ_zip_df.drop(econ_zip_df.index[99138:99142])

In [150]:
econ_zip_df.Description.unique()

array(['Personal income (thousands of dollars)',
       ' Net earnings by place of residence',
       ' Personal current transfer receipts',
       '  Income maintenance benefits 1/',
       '  Unemployment insurance compensation', '  Retirement and other',
       ' Dividends, interest, and rent 2/', ' Population (persons) 3/',
       ' Per capita personal income 4/', ' Per capita net earnings 4/',
       ' Per capita personal current transfer receipts 4/',
       '  Per capita income maintenance benefits 4/',
       '  Per capita unemployment insurance compensation 4/',
       '  Per capita retirement and other 4/',
       ' Per capita dividends, interest, and rent 4/',
       ' Earnings by place of work', '  Wages and salaries',
       '  Supplements to wages and salaries',
       '   Employer contributions for employee pension and insurance funds 5/',
       '   Employer contributions for government social insurance',
       "  Proprietors' income", "   Farm proprietors' income",
  

In [151]:
UniqueDesc = econ_zip_df.Description.unique()

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in UniqueDesc}

for key in DataFrameDict.keys():
    DataFrameDict[key] = econ_zip_df[:][econ_zip_df.Description == key]
    
DataFrameDict['Personal income (thousands of dollars)']

,GeoFIPS,GeoName,Region,Description,Unit,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"""00000""",United States,,Personal income (thousands of dollars),Thousands of dollars,12541995000,13315478000,13998383000,14175503000,14982715000,15717140000,16151881000,16937582000,17839255000,18542262000
31,"""01000""",Alabama,5,Personal income (thousands of dollars),Thousands of dollars,161516561,167942757,172804075,174415420,180457683,188711443,192281933,199999756,208752683,216449038
62,"""01001""","Autauga, AL",5,Personal income (thousands of dollars),Thousands of dollars,1826597,1896347,1927064,1944878,2019288,2131416,2203281,2276561,2360366,2453617
93,"""01003""","Baldwin, AL",5,Personal income (thousands of dollars),Thousands of dollars,6618292,7067055,7274734,7448839,7887975,8444952,8956578,9471242,10065966,10600260
124,"""01005""","Barbour, AL",5,Personal income (thousands of dollars),Thousands of dollars,758882,770008,766319,810596,799273,838910,824046,838184,872189,882834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98983,"""94000""",Plains,4,Personal income (thousands of dollars),Thousands of dollars,814191155,874230144,924061444,934446275,975636789,1009285884,1022335625,1055029341,1107629838,1146514617
99014,"""95000""",Southeast,5,Personal income (thousands of dollars),Thousands of dollars,2865792540,3020197130,3157295131,3170413258,3353410112,3528554087,3632818811,3818754861,4022275849,4173677027
99045,"""96000""",Southwest,6,Personal income (thousands of dollars),Thousands of dollars,1388316272,1504832700,1602000506,1642078939,1760489862,1802899581,1807841234,1924648064,2051026751,2144763946
99076,"""97000""",Rocky Mountain,7,Personal income (thousands of dollars),Thousands of dollars,406178027,438915419,467719020,485557545,522128637,550687735,565286733,602994187,647600678,680390074


In [162]:
econ_zip_df[DataFrameDict['Personal income (thousands of dollars)']['GeoFIPS'] == 10016]

KeyError: False

In [35]:
# gb = econ_zip_df.groupby('Description')    
# [gb.get_group(x) for x in gb.groups]

[        GeoFIPS         GeoName Region TableName  LineCode  \
 18      "00000"   United States          CAINC30     201.0   
 49      "01000"         Alabama      5   CAINC30     201.0   
 80      "01001"     Autauga, AL      5   CAINC30     201.0   
 111     "01003"     Baldwin, AL      5   CAINC30     201.0   
 142     "01005"     Barbour, AL      5   CAINC30     201.0   
 ...         ...             ...    ...       ...       ...   
 99001   "94000"          Plains      4   CAINC30     201.0   
 99032   "95000"       Southeast      5   CAINC30     201.0   
 99063   "96000"       Southwest      6   CAINC30     201.0   
 99094   "97000"  Rocky Mountain      7   CAINC30     201.0   
 99125   "98000"        Far West      8   CAINC30     201.0   
 
       IndustryClassification  \
 18                       ...   
 49                       ...   
 80                       ...   
 111                      ...   
 142                      ...   
 ...                      ...   
 99001     

In [163]:
# econ_zip_df['GeoFIPS'] = econ_zip_df['GeoFIPS'].str.replace('"', '')
econ_zip_df['GeoFIPS'] = pd.to_numeric(econ_zip_df['GeoFIPS'].str.replace('"', ''))

In [166]:
test = econ_zip_df.loc[econ_zip_df['GeoFIPS'].isin(zipcodes)]

In [167]:
test

,GeoFIPS,GeoName,Region,Description,Unit,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
62,1001,"Autauga, AL",5,Personal income (thousands of dollars),Thousands of dollars,1826597,1896347,1927064,1944878,2019288,2131416,2203281,2276561,2360366,2453617
63,1001,"Autauga, AL",5,Net earnings by place of residence,Thousands of dollars,1190297,1224272,1246525,1248619,1286995,1346348,1395861,1435492,1489045,1555278
64,1001,"Autauga, AL",5,Personal current transfer receipts,Thousands of dollars,368849,391827,390405,403133,423992,465280,472592,501554,521491,542039
65,1001,"Autauga, AL",5,Income maintenance benefits 1/,Thousands of dollars,41957,44832,43347,44264,44190,53078,51131,49377,48821,51336
66,1001,"Autauga, AL",5,Unemployment insurance compensation,Thousands of dollars,11248,9636,6747,4866,2730,2343,2354,2011,1887,1831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98358,56011,"Crook, WY",7,Farm proprietors employment 6/,Number of jobs,419,427,418,415,427,429,428,424,423,427
98359,56011,"Crook, WY",7,Nonfarm proprietors employment,Number of jobs,1281,1387,1442,1394,1405,1461,1571,1640,1536,1566
98360,56011,"Crook, WY",7,Average earnings per job (dollars),Dollars,30952,33782,34313,38370,41388,40092,37297,38199,39766,40254
98361,56011,"Crook, WY",7,Average wages and salaries,Dollars,34690,35194,36494,38088,39240,40133,40491,42006,44467,45542


In [142]:
econ_zip_df = econ_zip_df.iloc[econ_zip_df['GeoFIPS'].isin(zipcodes)]

NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [139]:
econ_zip_df['GeoFIPS'].isin(zipcodes)

62       True
63       True
64       True
65       True
66       True
         ... 
98358    True
98359    True
98360    True
98361    True
98362    True
Name: GeoFIPS, Length: 22940, dtype: bool

In [129]:
econ_zip_df[econ_zip_df['Description'] == ' Per capita dividends, interest, and rent 4/']

,GeoFIPS,GeoName,Region,Description,Unit,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
76,1001,"Autauga, AL",5,"Per capita dividends, interest, and rent 4/",Dollars,4883,5074,5280,5356,5616,5829,6061,6130,6299,6377
138,1005,"Barbour, AL",5,"Per capita dividends, interest, and rent 4/",Dollars,4282,4886,4646,4530,4768,5129,5158,5234,5512,5635
169,1007,"Bibb, AL",5,"Per capita dividends, interest, and rent 4/",Dollars,2773,2853,2873,2849,3047,3100,3194,3302,3395,3474
262,1013,"Butler, AL",5,"Per capita dividends, interest, and rent 4/",Dollars,3855,4167,4695,4681,4837,5085,5264,5852,5665,5785
479,1027,"Clay, AL",5,"Per capita dividends, interest, and rent 4/",Dollars,3906,3976,4093,4115,4369,4747,4997,4895,4880,4992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97912,55129,"Washburn, WI",3,"Per capita dividends, interest, and rent 4/",Dollars,6322,7596,8887,8151,8943,9186,9216,9935,10083,10263
98129,55901,"Shawano (includes Menominee), WI*",3,"Per capita dividends, interest, and rent 4/",Dollars,(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA),(NA)
98191,56001,"Albany, WY",7,"Per capita dividends, interest, and rent 4/",Dollars,7566,9571,11032,9539,9528,9449,9032,9140,10142,10306
98222,56003,"Big Horn, WY",7,"Per capita dividends, interest, and rent 4/",Dollars,5490,6395,7288,6104,6491,6518,6288,6642,7346,7510


In [133]:
econ_zip_df_melted = pd.melt(econ_zip_df, id_vars=['GeoFIPS', 'GeoName', 
                     'Description', 'Unit'],\
        value_vars=['2010', '2011', '2012', 
                    '2013', '2014', '2015', 
                    '2016', '2017', '2018', 
                    '2019'])

In [134]:
grouped_by_descr_melted = econ_zip_df_melted.groupby('Description')
grouped_by_area_melted = econ_zip_df_melted.groupby('GeoName')
grouped_by_zip_melted = econ_zip_df_melted.groupby('GeoFIPS')

In [135]:
grouped_by_descr_melted.get_group('Personal income (thousands of dollars)')

,GeoFIPS,GeoName,Description,Unit,variable,value
0,1001,"Autauga, AL",Personal income (thousands of dollars),Thousands of dollars,2010,1826597
31,1005,"Barbour, AL",Personal income (thousands of dollars),Thousands of dollars,2010,758882
62,1007,"Bibb, AL",Personal income (thousands of dollars),Thousands of dollars,2010,573063
93,1013,"Butler, AL",Personal income (thousands of dollars),Thousands of dollars,2010,596826
124,1027,"Clay, AL",Personal income (thousands of dollars),Thousands of dollars,2010,386881
...,...,...,...,...,...,...
229245,55129,"Washburn, WI",Personal income (thousands of dollars),Thousands of dollars,2019,777449
229276,55901,"Shawano (includes Menominee), WI*",Personal income (thousands of dollars),Thousands of dollars,2019,(NA)
229307,56001,"Albany, WY",Personal income (thousands of dollars),Thousands of dollars,2019,1660085
229338,56003,"Big Horn, WY",Personal income (thousands of dollars),Thousands of dollars,2019,463561


In [141]:
grouped_by_zip_melted.get_group('10025')

KeyError: '10025'